## Importing relevant libraries

In [31]:
import numpy as np
from typing import Tuple
import scipy.linalg as la
import scipy.spatial as spat
from scipy.stats import unitary_group
from scipy.stats import moment
from scipy.stats import skew, kurtosis
from scipy.optimize import curve_fit
from scipy.linalg import norm
import matplotlib.pyplot as plt
import math
from dataclasses import dataclass
import time
from datetime import datetime

# Libraries for implementing the VQD algorithm
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector, Gate
from qiskit.primitives import Sampler, Estimator
from qiskit_aer import AerSimulator
from qiskit_algorithms.utils import algorithm_globals
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session, Options
from qiskit_ibm_runtime import Estimator as EstimatorNew
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import SparsePauliOp, partial_trace, entropy
from qiskit.circuit.library import RealAmplitudes, TwoLocal, EfficientSU2
from qiskit_algorithms.optimizers import *
from qiskit_algorithms.state_fidelities import ComputeUncompute

from qiskit_algorithms.eigensolvers import EigensolverResult, VQD
from qiskit_algorithms import NumPyMinimumEigensolver, VQE

# Import classical optimizers
from qiskit_algorithms.optimizers import SPSA, P_BFGS, COBYLA, IMFIL, SNOBFIT, NELDER_MEAD, SLSQP, NFT, ADAM, POWELL, GradientDescent, BOBYQA

# Import Statevector and SparsePauliOp
from qiskit.quantum_info import SparsePauliOp, Statevector

# Import noise models
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

# Import a fake backend and Qiskit simulators and/or noise libraries
from qiskit_aer import AerSimulator
# from qiskit_aer.primitives import Estimator as AerEstimator 
# from qiskit_aer.noise import NoiseModel

# Import the FakeManila backend
from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeMontrealV2, FakeGuadalupeV2, FakeManila

## Constructing the FPE operator

In [6]:
## Classical implementation
"""
Purpose: 
    Find the analytical solution given the input parameters
Input: 
    x: array representing the x coordinates over which the solution is to be found
    a, Γ: parameters for the Ornstein-Uhlenbeck equation
Output:
    y: array of the value of the analytical solution over x
"""

def OrnsteinUhlenbeck(x, a = 1, gamma = 1):
    y = np.sqrt(a/(2*np.pi*gamma)) * np.exp((-a * x ** 2)/(2*gamma))
    
    return y

"""
Purpose:
    Calculate the weighted Hermite polynomials using recursive relations up to H_n
Input:
    n: the highest number of Hermite polynomials to be evaluated at
    x: the value of x at which the weighted Hermite polynomials are to be evaluated
Output:
    Hermite: the value of the weighted Hermite polynomials
Example: HermiteSeries(2, 4) -> [H_0(4), H_1(4), H_2(4)]
"""

def HermiteSeries(n, x):
    Hermite = np.zeros((1, n+1))
    Hermite[0][0] = 1.0 * np.exp(-x**2/2)
    Hermite[0][1] = 2.0 * x *np.exp(-x**2/2)
    
    for ni in range(1, n):
        Hermite[0][ni+1] = 2*x*Hermite[0][ni] - 2*ni*Hermite[0][ni-1]
    
    return Hermite

"""
Purpose:
    Reconstruct the functions represented by the coefficients of weighted Hermite polynomials in spatial coordinates
Input:
    x: array containing the x values over which the function is to be evaluated
    coefficients: the coefficients corresponding to the Hermite polynomials (should be zero-mode of the operator)
    nmax: the highest order of weighted Hermite polynomials used
    s: the scaling factor
Output:
    y: the reconstruction
"""

def project(x, coefficients, nmax, s):
    
    y = np.zeros(x.shape)
    
    for i in (range(len(x))):
        x0 = s*x[i]
        hermite_values = HermiteSeries(nmax, x0)
        y[i] += np.dot(hermite_values, coefficients)
        
    return (x, y)

"""
Purpose:
    Normalize the PDF so that the area under the curve is 1
Input:
    x, y: arrays representing the unnormalized
Output:
    (x, y): normalized PDF
"""

def normalize(x, y):
    dx = x[1] - x[0]
    sum = 0
    
    for i in range(len(x)):
        sum = sum + (y[i] * dx)
    
    y = y/sum
    
    return (x, y)

"""
Purpose:
    Calculate the RMS
Input:
    y_op, y: the y values generated by the operator and the analytical solution
Output:
    RMS
"""

def rms_diff(y_op, y):
    return np.sqrt(np.sum((y_op-y)**2)/len(y))

"""
Purpose:
    Find s given a and Γ such that the projection in Hermite space is just H_0
Input:
    a, Γ: parameters for Ornstein-Uhlenbeck
Output:
    s
"""

def special_s(a, gamma):
    return np.sqrt(a/gamma)

def euler_representation(z):
    A = abs(z)
    phase = math.atan(z.imag/z.real)
    
    return A, phase

def generate_positive_semidefinite_matrix(eigenvalues):
    ## Function to transform the FPE operator to the Hermite basis and return a finite truncated matrix for the ...
    ## ... FPE operator
    
    n = len(eigenvalues)
    s = np.diag(eigenvalues)
    q, _ = la.qr(np.random.rand(n, n))
    semidef = np.dot(np.transpose(q), s)
    semidef = np.dot(semidef, q)
    
    return semidef

def normalize_probability(y, dx):
    ## Function to return a normalized PDF 
    
    total = np.sum(y) * dx
    y = y / total
    
    return y

def perturbed_ornstein_uhlenbeck(x0, a = 1, gamma = 1, c = 0, shift = 0):
    ## Function to compute the exact solution to the perturbed Ornstein-Uhlenbeck equation
    
    assert len(x0) > 1, "Error: x0 should be a vector"

    x = x0 - shift
    y = np.exp(-a*x**2 /(2*gamma) - c*x**4 /(4*gamma))
    dx = x[2] - x[1]
    
    y = normalize_probability(y, dx)
    
    return y

def special_L(a, gamma):
    ## Function to compute the characteristic length scale
    
    return np.sqrt(a/gamma)

def delta(N, n):
    ## Function to implement the Kronecker delta function
    
    if N == n:
        return 1
    else:
        return 0
    
def matrix_element(N, n, L):
    # Function to compute the matrix elements of the position operator
    
    value = L/np.sqrt(2) * (np.sqrt(n) * delta(N, n-1) + np.sqrt(n+1) * delta(N, n+1))
    
    return value

def create_position(nmax, L):
    # Function to construct the position operator
    
    op = np.zeros((nmax + 1, nmax + 1))
    
    for N in range(nmax+1):
        row = N
        for n in range(nmax+1):
            col = n
            op[row, col] = matrix_element(N, n, L)
            
    return op

def f(N,n):
    return 0.5*( np.sqrt(n*(n-1))*delta(N,n-2) + delta(N,n) - np.sqrt((n+1)*(n+2))*delta(N,n+2))

def g(N,n, L):
    return (0.5 / L**2)*( np.sqrt(n*(n-1))*delta(N,n-2) - (2*n+1)*delta(N,n) + np.sqrt((n+1)*(n+2))*delta(N,n+2))

def t(N, n, L):
    val1 = np.sqrt(n*(n-1)*(n-2)*(n-3))*delta(N, n-4)
    val2 = (2*n+2)*np.sqrt(n*(n-1))*delta(N, n-2)
    val3 = (6*n+3)*delta(N, n)
    val4 = -2*n*np.sqrt((n+1)*(n+2))*delta(N, n+2)
    val5 = -1*np.sqrt((n+1)*(n+2)*(n+3)*(n+4))*delta(N, n+4)

    return (L**2/4)*( val1 + val2 + val3 + val4 + val5 )

def element_perturbed(N, n, L, a, c, gamma):
    ## Function to compute the matrix elements of the finite truncated matrix for the FPE operator (in the Hermite basis)
    
    return -(a*f(N,n) + c*t(N, n, L) + gamma*g(N, n, L))

def create_operator_perturbed(nmax, L, a, c, gamma):
    ## Function to generate the FPE operator matrix in the Hermite basis
    op = np.zeros((nmax+1, nmax+1))
    
    for N in range(nmax+1):
        row = N
        for n in range(nmax+1):
            col = n
            op[row, col] = element_perturbed(N, n, L, a, c, gamma)
    
    """
    if np.any(np.nan, op):
        print("There's an nan in the operator")
    elif np.any(np.inf, op):
        print("There is an inf in the operator")
    """

    return op

def state_n(nmax, x0, L):
    ## Function to compute the Hermite polynomials upto the order specified by nmax
    
    assert L > 0, "Error (state_n): input L must be greater than or equal to 0"

    states = np.zeros(nmax + 1)
    x = x0 / L

    states[0] = np.sqrt(L / np.sqrt(1/np.pi))* 1.0 * np.exp(-x**2/2)

    if nmax > 0:
        states[1] = np.sqrt(L / np.sqrt(1/np.pi))* (1/np.sqrt(2)) * 2.0 * x * np.exp(-(x**2)/2)
    
    for ni in range(2, nmax + 1):
        states[ni] = ((np.sqrt(2)*x) / np.sqrt(ni))*states[ni-1] - (ni-1)/np.sqrt((ni)*(ni-1))*states[ni-2]

    return states

def integrate_eigenvector(x0, y, nmax, L):
    ## Function to compute the coefficients corresponding to the calculated Hermite polynomials by numerical integration
    ## Note: here, we use the left-hand integration method
    
    dx = x0[2] - x0[1]
    eigenvector = np.zeros(nmax + 1)

    for i in range(len(x)):

        states = state_n(nmax, x0[i], L)
        states = states * y[i] * dx

        eigenvector = eigenvector + states
        
    return {"eigenvector" : eigenvector, "dx" : dx, "nmax" : nmax, "x0" : x0, "L" : L}

def make_operator_even(op):
    op_new = np.zeros((op.shape[0]//2, op.shape[1]//2))
    
    for row in range(op_new.shape[0]):
        for col in range(op_new.shape[1]):
            op_new[row, col] = op[row*2, col * 2]
    
    return op_new

def reconstruct_eigenvector(cache, normalize = True, only_even = False):
    ## Function to reconstruct the PDF using the cache obtained previously
    
    eigenvector = cache["eigenvector"]
    nmax = cache["nmax"]
    x0 = cache["x0"]
    dx = cache["dx"]
    L = cache["L"]
    
    if not only_even:
        eigenvector = cache["eigenvector"]
    else:
        eigenvector_old = cache["eigenvector"]
        eigenvector = np.zeros(nmax + 1)
        
        for i in range(len(eigenvector_old)):
            eigenvector[2*i] = eigenvector_old[i]

    y = np.zeros(len(x0))

    for i in range(len(x0)):
        states = state_n(nmax, x0[i], L)
        y[i] += (np.dot(states, eigenvector))
    if normalize:   
        y = normalize_probability(y, dx)
        
    return (x0, y)

def find_zeromode(op_nonhermitian, nmax, x0, dx, L, which = "single", only_even = False):
    ## Function to compute the zeromode, its position, and the eigenvalue corresponding to the zeromode
    
    assert which == "nonhermitian" or which == "single", "Error: currently only supports which = {\"nonhermitian\", \"single\"}"
    
    if only_even:
        op_processed = make_operator_even(op_nonhermitian)
    else:
        op_processed = op_nonhermitian
        
    if which == "nonhermitian":
        op = op_processed
    elif which == "single":
        op = np.dot(np.transpose(op_processed), op_processed)

    eigenvalues, eigenvectors = la.eig(op)

    index = np.argmin(np.real(eigenvalues)**2)

    min_eigenval = eigenvalues[index]
    
    zeromode = eigenvectors[:, index]

    return {"operator" : op, "which" : which, \
            "eigenvector" : np.real(zeromode), "index" : index, "eigenvalue" : np.real(min_eigenval), \
            "nmax" : nmax, "x0" : x0, "dx" : dx, "L" : L, "spectrum" : eigenvalues, "P" : eigenvectors}

def rms_diff(y0, y, sigdigits = 0):
    ## Function to compute the RMS difference between the analytically computed PDF and the one ...
    ## ... obtained using the FPE operator
    
    assert len(y0) == len(y), "Error: the length of y0 and y should be the same"
    diff = np.sum((y0-y)**2)/len(y0)
    
    return np.sqrt(diff)

def analyze_collapse(initial_state, P, index):
    num_basis = P.shape[1]
    coefficients = np.zeros((num_basis, ))

    for i in range(num_basis):
        basis = P[:, i]
        basis = normalize_eigenvector(basis)

        coefficients[i] = np.dot(basis, initial_state)

    normalized_coefficients = normalize_eigenvector(coefficients)

    reconstruction = np.zeros((P.shape[0], ))

    for i in range(num_basis):
        basis = P[:, i]
        basis = normalize_eigenvector(basis)

        added_vector = coefficients[i] * basis
        reconstruction = reconstruction + added_vector

    probability = normalized_coefficients ** 2
    
    probability_zeromode = probability[index]
    assert np.sum(probability) - 1 < 0.00001, "Error: unity normalization of probability is not observed"

    print("The initial_state is: ")
    print(initial_state)
    print()

    print("The coefficient expansion is: ")
    print(coefficients)
    print()

    print("The normalized coefficient expansion is: ")
    print(normalized_coefficients)
    print()

    print("The reconstructed vector is: ")
    print(reconstruction)
    print()

    print("The probability of collapsing into each eigenvector is: ")
    print(probability)
    print()

    print("The probability of collapsing into the zeromode is: ")
    print(probability_zeromode)
    print()
    
    return probability_zeromode, probability

def get_unitary(matrix, add_half = False):
    """
    Purpose: given a matrix, returns the unitary, hermitian matrix to be diagonalized
    Input: matrix -> the matrix to be diagonalized
    Output: U -> the unitary matrix
            nqubits -> the number of qubis needed to represent the basis of U
            dimension -> the dimension of the original matrix
    """
    assert matrix.ndim == 2, "Error: Only a matrix maybe processed"
    assert matrix.shape[0] == matrix.shape[1], "Error: Only a square matrix maybe processed"

    if np.any(np.transpose(matrix) != matrix):
        matrix_T = np.transpose(matrix)
        matrix = np.dot(matrix_T, matrix)

    ## Finding the dimension of the matrix
    dimension_hermitian = matrix.shape[0]

    ## Finding the number of qubits required to represent the matrix
    nqubits = int(np.ceil(np.log2(dimension_hermitian)))

    ## Construct the relevant matrix
    op_dim = 2 ** nqubits
    op = np.eye(op_dim)
    op[0:dimension_hermitian, 0:dimension_hermitian] = np.copy(matrix)

    if add_half:
        op = op + np.pi * np.eye(op.shape[0])

    U = la.expm(1j*op)

    # Get the dimensions of the unitary matrix
    dimension = U.shape[0]

    return U, nqubits, dimension

def expect_value(zeromode, matrix):
    
    value = np.dot(matrix, zeromode)
    expect = np.dot(np.transpose(zeromode), value)
    
    return expect

## Helper functions

In [9]:
def find_probability(eigenvector_raw):
    """
    Purpose: Find the probability associated with each basis of an eigenvector
    Input: eigenvector_raw -> Numpy array documenting the number of times each basis is detected within the eigenvector
    Output: eigenvector_prob -> Numpy array documenting the probability of detecting each basis
    """
    count_total = np.sum(eigenvector_raw)
    eigenvector_prob = eigenvector_raw / count_total
    
    return eigenvector_prob

def find_amplitude(eigenvector_prob):
    """
    Purpose: Finding the probability amplitude of each basis using quantum mechanics
    Input: eigenvector_prob -> Numpy array documenting the probability that each basis is measured
    Output: eigenvector -> Numpy array representing the eigenvector
    """
    eigenvector = np.sqrt(eigenvector_prob)
    return eigenvector

def normalize_eigenvector(vector):
    """
    Purpose: Normalizes a vector such that its norm is 1
    Input: vector -> The vector to be normalized
    Output: vector -> The normalized vector
    """
    L2 = np.sum(np.square(vector))
    vector = vector / np.sqrt(L2)

    return vector

def make_operator_even(op):
    op_new = np.zeros((op.shape[0]//2, op.shape[1]//2))

    for row in range(op_new.shape[0]):
        for col in range(op_new.shape[1]):
            op_new[row, col] = op[row*2, col * 2]

    return op_new

def get_pdf(n, x, dx, L, shift, zeromode_qpe, normalize = True, make_even = False):
    # Function to construct the ground state PDF using the VQSVD zeromode
    
    if not make_even:
        eigenvector = zeromode_qpe
    else:
        eigenvector_old = zeromode_qpe
        eigenvector = np.zeros(n + 1)
        for i in range(len(eigenvector_old)):
            eigenvector[2*i] = eigenvector_old[i]
            
    x0 = x - shift
    
    # Computing the PDF
    y = np.zeros(len(x0))

    for i in range(len(x0)):
        states = state_n(nmax, x0[i], L)
        y[i] += (np.dot(states, eigenvector))
    
    if normalize:
        y = normalize_probability(y, dx)

    return x0, y

def compute_expectation_x_squared_simpson(x, y, n):
    """
    Computes the expectation value of x^2 using Simpson's rule for numerical integration.
    
    Parameters:
    x (array-like): Discrete values of x.
    y (array-like): Corresponding values of the probability density function (PDF) at x.
    
    Returns:
    float: The expectation value of x^2.
    """
    # Ensure x and y are numpy arrays
    x = np.array(x)
    y = np.array(y)
    
    # Compute x^2
    x_squared = x**n
    
    # Check if the number of intervals is even, if not make it even by truncating the last point
    if len(x) % 2 == 0:
        x = x[:-1]
        y = y[:-1]
        x_squared = x_squared[:-1]
    
    # Compute the integral using Simpson's rule
    h = (x[-1] - x[0]) / (len(x) - 1)
    integral = y[0] * x_squared[0] + y[-1] * x_squared[-1] + \
               4 * np.sum(y[1:-1:2] * x_squared[1:-1:2]) + \
               2 * np.sum(y[2:-2:2] * x_squared[2:-2:2])
    integral *= h / 3
    
    return integral

def get_pdf(n, x, dx, L, shift, zeromode_qpe, normalize = True, make_even = False):
    # Function to construct the ground state PDF using the VQSVD zeromode
    
    if not make_even:
        eigenvector = zeromode_qpe
    else:
        eigenvector_old = zeromode_qpe
        eigenvector = np.zeros(n + 1)
        for i in range(len(eigenvector_old)):
            eigenvector[2*i] = eigenvector_old[i]
            
    x0 = x - shift
    
    # Computing the PDF
    y = np.zeros(len(x0))

    for i in range(len(x0)):
        states = state_n(nmax, x0[i], L)
        y[i] += (np.dot(states, eigenvector))
    
    if normalize:
        y = normalize_probability(y, dx)

    return x0, y

# Fidelity measure 1
def get_fidelity(zeromode_classic, zeromode_vqe):
    # Function to compute the infidelity

    overlap = np.dot(np.transpose(zeromode_vqe), zeromode_classic)
    fidelity = 1 - overlap ** 2
    return fidelity

# Fidelity measure 2
def get_similarity(a, b):
    # Function to compute the similarity between 2 zeromodes
    
    numerator = np.abs(np.dot(a.conj().T, b))**2
    denominator = np.linalg.norm(a)**2 * np.linalg.norm(b)**2
    
    return numerator / denominator

def compute_errors(expect_classical, expect_quantum):

    error = np.abs(expect_classical - expect_quantum) / expect_classical
    return error

## VQE implementation

In [12]:
# VQE run for a given ansatz and optimizer
def run_vqe_ansatz_analysis(matrix, ansatz, optimizer, seed, exact_ground_state, classical_expectation, nmax, L):

    # Define parameters to compute <x^2>
    dx = 0.01
    x = np.linspace(-4, 4, int(8/dx))

    # Get the Pauli-decomposed form of the operator
    qub_hamiltonian = SparsePauliOp.from_operator(matrix)
    dimension = matrix.shape[0]
    num_qubits = int(np.log2(dimension))

    # Set up the random initial point
    np.random.seed(seed)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)

    # Initialize the Estimator primitive
    estimator = Estimator()

    # Logging class for VQE callback
    @dataclass
    class VQELog:
        parameters: list
        values: list
        def update(self, count, parameters, mean, _metadata):
            self.values.append(mean)
            self.parameters.append(parameters)
            
    log = VQELog([], [])
    
    # Run VQE with the given ansatz and optimizer
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point, callback=log.update)

    # Get the VQE results
    result = vqe.compute_minimum_eigenvalue(qub_hamiltonian)

    # Get the number of optimizer function calls
    num_calls = result.cost_function_evals

    # Extract the optimal parameters and construct the state vector
    optimal_params = result.optimal_point
    final_circuit = ansatz.assign_parameters(optimal_params)
    vqe_statevector = Statevector.from_instruction(final_circuit)

     # Convert the quantum and classical zeromodes into 4 x 1 arrays
    exact_ground_state = np.array(exact_ground_state).reshape((len(exact_ground_state), 1))
    vqe_statevector = vqe_statevector.data.tolist()

    if len(exact_ground_state) == 6:
        vqe_statevector = vqe_statevector[:6]
        zeromode = np.array(vqe_statevector).reshape((len(exact_ground_state), 1))
        zeromode = np.real(zeromode)
    
        # Compute the fidelity measure
        fidelity_value = get_similarity(exact_ground_state, zeromode)
    
    else:
        zeromode = np.array(vqe_statevector).reshape((len(exact_ground_state), 1))
        zeromode = np.real(zeromode)
    
        # Compute the fidelity measure
        fidelity_value = get_similarity(exact_ground_state, zeromode)

    # Compute <x^2>
    x_quantum, y_quantum = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
    quantum_expectation = compute_expectation_x_squared_simpson(x_quantum, y_quantum, 2)
    error = compute_errors(classical_expectation, quantum_expectation)
        
    return zeromode, fidelity_value, quantum_expectation, error, num_calls

## Get the matrix and the zeromode

In [15]:
def get_zeromode(nmax, a, c, L, gamma):
    # Function to construct the matrix and get the zeromode

    #3, 5, 1, 1
    dx = 0.01
    x = np.linspace(-4, 4, int((8)/dx))
    
    ## Finding the zeromode through diagonalization
    op_nonhermitian = create_operator_perturbed(nmax, L, a, c, gamma)
    only_even = True
    
    # Matrix
    cache_diagonalization = find_zeromode(op_nonhermitian, nmax, x, dx, L, which = "single", only_even = only_even)
    matrix = cache_diagonalization['operator']

    # Get the classical zeromode
    A, P = la.eig(matrix)
    
    # Get the zeromode
    A_real = np.real(A)
    index = np.where(A_real == np.amin(A_real))[0][0]
    
    eigenvalue = A[index]
    zeromode_classic = P[:, index]
    
    zeromode_classic = np.real(normalize_eigenvector(zeromode_classic))
    zeromode_classic = np.reshape(zeromode_classic, (zeromode_classic.size, 1))

    return matrix, zeromode_classic

 ## Estimating the standard deviation in results

In [94]:
def performance_metrics(matrix, zeromode_classic, pair_depth_dict, classical_expectation, nmax, L):
    """
    Estimate VQE resource usage and relative error statistics for given optimizer-ansatz pairs at fixed depths.
    
    Parameters:
    - matrix: Hamiltonian matrix
    - zeromode_classic: exact classical solution
    - pair_depth_dict: dict of form {'SLSQP-RealAmplitudes': 2, 'P_BFGS-EfficientSU2': 3, ...}
    - classical_expectation: expectation operator for computing observable (like <x^2>)
    - nmax, L: problem parameters

    Returns:
    - error_stats: dict with mean and std of relative error for each pair
    - resource_info: dict with function call count, gate count, and circuit depth for each pair
    """

    # Get the qubit number 
    dimension = matrix.shape[0]
    num_qubits = int(np.log2(dimension))
    
    # Mapping of names to classes
    optimizer_map = {
        "SLSQP": SLSQP,
        "P_BFGS": P_BFGS
    }

    ansatz_map = {
        "RealAmplitudes": RealAmplitudes,
        "TwoLocal": TwoLocal,
        "EfficientSU2": EfficientSU2,
    }

    # Initialize
    expectation_stats = {}

    for pair_name, depth in pair_depth_dict.items():
        print(f"\nRunning VQE for: {pair_name} with depth {depth}")

        # Split pair_name to get optimizer and ansatz
        opt_name, ansatz_name = pair_name.split('-')
        optimizer_class = optimizer_map[opt_name]
        AnsatzClass = ansatz_map[ansatz_name]

        # Instantiate optimizer with correct stopping condition
        if opt_name == "P_BFGS":
            optimizer = optimizer_class(maxfun=5000)
        else:
            optimizer = optimizer_class(maxiter=5000)

        # Initialize ansatz
        if AnsatzClass == RealAmplitudes:
            ansatz = AnsatzClass(num_qubits=num_qubits, entanglement='full', reps=depth)
        elif AnsatzClass == TwoLocal:
            ansatz = AnsatzClass(num_qubits=num_qubits, rotation_blocks=['ry'],
                                 entanglement_blocks='cx', reps=depth)
        elif AnsatzClass == EfficientSU2:
            ansatz = AnsatzClass(num_qubits=num_qubits, su2_gates=['ry'],
                                 entanglement='sca', reps=depth)

        # Run 10 independent VQE runs
        all_quantum_expectations = []
        all_function_calls = []

        for run in range(10):
            seed = run + 1

            zeromode, fidelity_value, quantum_expectation, error, function_call_count = run_vqe_ansatz_analysis(
                matrix=matrix,
                ansatz=ansatz,
                optimizer=optimizer,
                seed=seed,
                exact_ground_state=zeromode_classic,
                classical_expectation=classical_expectation,
                nmax=nmax,
                L=L
            )

            all_quantum_expectations.append(quantum_expectation)
            all_function_calls.append(function_call_count)

        # Compute stats
        std_expectation = np.std(all_quantum_expectations)

        # Store results
        expectation_stats[pair_name] = {
            'std_expectation': std_expectation
        }

        print(f"{pair_name} — Std Dev: {std_expectation:.5f}")

    return expectation_stats

################# Might not be of any use to us #############################################################
def compute_entanglement_entropies(n_qubits: int,
                                    reps_real: int = 4,
                                    reps_two: int = 1,
                                    reps_eff: int = 3,
                                    seed: int = 42, 
                                   ) -> Tuple[float, float, float]:

    # Initialize a random seed
    rng = np.random.default_rng(seed)

    # Define circuits
    circ_real = RealAmplitudes(n_qubits, entanglement='full', reps=reps_real)
    circ_two = TwoLocal(n_qubits, rotation_blocks='ry', entanglement_blocks='cx', reps=reps_two)
    circ_eff = EfficientSU2(n_qubits, su2_gates=['ry'],
                                 entanglement='sca', reps=reps_eff)

     # Sort parameters to bind them consistently
    def assign_sorted_parameters(circuit, rng):
        sorted_params = sorted(circuit.parameters, key=lambda p: p.name)
        param_values = rng.uniform(-np.pi, np.pi, len(sorted_params))
        param_map = dict(zip(sorted_params, param_values))
        return circuit.assign_parameters(param_map)

    circ_real = assign_sorted_parameters(circ_real, rng)
    circ_two = assign_sorted_parameters(circ_two, rng)
    circ_eff = assign_sorted_parameters(circ_eff, rng)

    # # Bind random values to all parameters
    # circ_real = circ_real.assign_parameters(np.random.uniform(-np.pi, np.pi, circ_real.num_parameters))
    # circ_two = circ_two.assign_parameters(np.random.uniform(-np.pi, np.pi, circ_two.num_parameters))
    # circ_eff = circ_eff.assign_parameters(np.random.uniform(-np.pi, np.pi, circ_eff.num_parameters))

    # Compute statevectors
    state_real = Statevector.from_instruction(circ_real)
    state_two = Statevector.from_instruction(circ_two)
    state_eff = Statevector.from_instruction(circ_eff)

    # Trace out second half of the qubits
    trace_indices = list(range(n_qubits // 2, n_qubits))
    red_real = partial_trace(state_real, trace_indices)
    red_two = partial_trace(state_two, trace_indices)
    red_eff = partial_trace(state_eff, trace_indices)

    # Compute entropies
    S_real = entropy(red_real, base=2)
    S_two = entropy(red_two, base=2)
    S_eff = entropy(red_eff, base=2)

    return S_real, S_two, S_eff

## Quantum (VQE) analysis 

### For a = 1, b = 2 (N = 8)

### Get the matrix and the zeromode

In [62]:
## Get the matrix and zeromode
nmax = 15
a = 1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 6.14062500e+00 -2.34671063e+01  3.55176013e+01 -9.22378041e+00
  -3.13747510e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.34671063e+01  9.68906250e+01 -1.51662699e+02  1.11470288e+02
  -6.65558976e+01 -2.10468228e+01  0.00000000e+00  0.00000000e+00]
 [ 3.55176013e+01 -1.51662699e+02  4.02390625e+02 -3.54479193e+02
   1.63951212e+02 -2.30876387e+02 -6.98044143e+01  0.00000000e+00]
 [-9.22378041e+00  1.11470288e+02 -3.54479193e+02  1.05164062e+03
  -5.71772019e+02  1.15363556e+02 -5.86246269e+02 -1.71932490e+02]
 [-3.13747510e+00 -6.65558976e+01  1.63951212e+02 -5.71772019e+02
   2.23364062e+03 -7.19516989e+02 -1.36244266e+02 -1.24067405e+03]
 [ 0.00000000e+00 -2.10468228e+01 -2.30876387e+02  1.15363556e+02
  -7.19516989e+02  4.19739062e+03 -6.89706553e+02 -7.16860080e+02]
 [ 0.00000000e+00  0.00000000e+00 -6.98044143e+01 -5.86246269e+02
  -1.36244266e+02 -6.89706553e+02  6.56939062e+03 -1.15978184e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.7

C:\Users\Owner\AppData\Local\Temp\ipykernel_51032\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Estimating the standard deviation in results (fix at 5000 optimization iterations)

In [68]:
# Initialize the optimizer -- ansatz pairings and corresponding depths
pair_depth_dict = {
    'SLSQP-RealAmplitudes': 4,
    'SLSQP-TwoLocal': 3,
    'SLSQP-EfficientSU2': 4, 
    'P_BFGS-RealAmplitudes': 4, 
    'P_BFGS-TwoLocal': 2,
    'P_BFGS-EfficientSU2': 6
}

# Run VQE
std_expectations_N_8 = performance_metrics(matrix, zeromode, pair_depth_dict, classical_expectation, nmax, L)


Running VQE for: SLSQP-RealAmplitudes with depth 4


C:\Users\Owner\AppData\Local\Temp\ipykernel_51032\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


SLSQP-RealAmplitudes — Std Dev: 0.00004

Running VQE for: SLSQP-TwoLocal with depth 3
SLSQP-TwoLocal — Std Dev: 0.00007

Running VQE for: SLSQP-EfficientSU2 with depth 4


For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 — Std Dev: 0.00004

Running VQE for: P_BFGS-RealAmplitudes with depth 4


...

P_BFGS-RealAmplitudes — Std Dev: 0.00000

Running VQE for: P_BFGS-TwoLocal with depth 2


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-TwoLocal — Std Dev: 0.00217

Running VQE for: P_BFGS-EfficientSU2 with depth 6


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 — Std Dev: 0.00000


### Print the performance metrics

In [70]:
## Print the mean and standard deviation errors
print('The standard deviation in the quantum expectation values are:')
print(std_expectations_N_8)
print()

The standard deviation in the quantum expectation values are:
{'SLSQP-RealAmplitudes': {'std_expectation': 3.510864439677807e-05}, 'SLSQP-TwoLocal': {'std_expectation': 7.220756463975331e-05}, 'SLSQP-EfficientSU2': {'std_expectation': 4.0110900607155564e-05}, 'P_BFGS-RealAmplitudes': {'std_expectation': 7.341360514253251e-07}, 'P_BFGS-TwoLocal': {'std_expectation': 0.0021679162648602945}, 'P_BFGS-EfficientSU2': {'std_expectation': 7.094362371700933e-07}}



### For a = 1, b = 2 (N = 16)

### Get the matrix and the zeromode

In [72]:
## Get the matrix and zeromode
nmax = 31
a = 1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 6.14062500e+00 -2.34671063e+01  3.55176013e+01 -9.22378041e+00
  -3.13747510e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.34671063e+01  9.68906250e+01 -1.51662699e+02  1.11470288e+02
  -6.65558976e+01 -2.10468228e+01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.55176013e+01 -1.51662699e+02  4.02390625e+02 -3.54479193e+02
   1.63951212e+02 -2.30876387e+02 -6.98044143e+01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-9.22378041e+00  1.11470288e+02 -3.54479193e+02  1.05164062e+03
  -5.71772019e+02  1.15363556e+02 -5.86246269e+02 -1.71932490e+02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000

C:\Users\Owner\AppData\Local\Temp\ipykernel_51032\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The value of <x^2> is:
0.3662333595845026



### Estimating the standard deviation in results (fix at 5000 optimization iterations)

In [74]:
# Initialize the optimizer -- ansatz pairings and corresponding depths
pair_depth_dict = {
    'SLSQP-RealAmplitudes': 4,
    'SLSQP-TwoLocal': 3,
    'SLSQP-EfficientSU2': 5, 
    'P_BFGS-RealAmplitudes': 4, 
    'P_BFGS-TwoLocal': 3,
    'P_BFGS-EfficientSU2': 5
}

# Run VQE
std_expectations_N_16 = performance_metrics(matrix, zeromode, pair_depth_dict, classical_expectation, nmax, L)


Running VQE for: SLSQP-RealAmplitudes with depth 4


C:\Users\Owner\AppData\Local\Temp\ipykernel_51032\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


SLSQP-RealAmplitudes — Std Dev: 0.00358

Running VQE for: SLSQP-TwoLocal with depth 3
SLSQP-TwoLocal — Std Dev: 0.00397

Running VQE for: SLSQP-EfficientSU2 with depth 5


For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 — Std Dev: 0.00381

Running VQE for: P_BFGS-RealAmplitudes with depth 4


...

P_BFGS-RealAmplitudes — Std Dev: 0.02066

Running VQE for: P_BFGS-TwoLocal with depth 3


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-TwoLocal — Std Dev: 0.00488

Running VQE for: P_BFGS-EfficientSU2 with depth 5


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 — Std Dev: 0.51767


### Print performance metrics

In [77]:
## Print the mean and standard deviation errors
print('The standard deviation in the quantum expectation values are:')
print(std_expectations_N_16)
print()

The standard deviation in the quantum expectation values are:
{'SLSQP-RealAmplitudes': {'std_expectation': 0.0035809261996627294}, 'SLSQP-TwoLocal': {'std_expectation': 0.003973431384293158}, 'SLSQP-EfficientSU2': {'std_expectation': 0.003809425699274018}, 'P_BFGS-RealAmplitudes': {'std_expectation': 0.020656127275909993}, 'P_BFGS-TwoLocal': {'std_expectation': 0.004876148039825101}, 'P_BFGS-EfficientSU2': {'std_expectation': 0.5176716115233391}}



### For a = -1, b = 2 (N = 8)

### Get the matrix and the zeromode

In [21]:
## Get the matrix and zeromode
nmax = 15
a = -1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 1.73906250e+01 -4.14983292e+01  4.16413256e+01 -2.51557647e+00
  -3.13747510e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-4.14983292e+01  1.38140625e+02 -2.04490248e+02  1.54161036e+02
  -3.10594189e+01 -2.10468228e+01  0.00000000e+00  0.00000000e+00]
 [ 4.16413256e+01 -2.04490248e+02  4.13640625e+02 -4.62654398e+02
   2.97161572e+02 -1.33665277e+02 -6.98044143e+01  0.00000000e+00]
 [-2.51557647e+00  1.54161036e+02 -4.62654398e+02  9.24890625e+02
  -7.68209032e+02  4.17083625e+02 -3.82334523e+02 -1.71932490e+02]
 [-3.13747510e+00 -3.10594189e+01  2.97161572e+02 -7.68209032e+02
   1.81289062e+03 -1.04918443e+03  4.35981651e+02 -8.73066927e+02]
 [ 0.00000000e+00 -2.10468228e+01 -1.33665277e+02  4.17083625e+02
  -1.04918443e+03  3.27864062e+03 -1.20958947e+03  2.51869758e+02]
 [ 0.00000000e+00  0.00000000e+00 -6.98044143e+01 -3.82334523e+02
   4.35981651e+02 -1.20958947e+03  4.90064062e+03 -1.53414981e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.7

C:\Users\Owner\AppData\Local\Temp\ipykernel_26540\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Estimating the standard deviation in results (fix at 5000 optimization iterations)

In [24]:
# Initialize the optimizer -- ansatz pairings and corresponding depths
pair_depth_dict = {
    'SLSQP-RealAmplitudes': 2,
    'SLSQP-TwoLocal': 2,
    'SLSQP-EfficientSU2': 4, 
    'P_BFGS-RealAmplitudes': 4, 
    'P_BFGS-TwoLocal': 2,
    'P_BFGS-EfficientSU2': 6
}

# Run VQE
std_expectations_N_8 = performance_metrics(matrix, zeromode, pair_depth_dict, classical_expectation, nmax, L)


Running VQE for: SLSQP-RealAmplitudes with depth 2


C:\Users\Owner\AppData\Local\Temp\ipykernel_26540\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]
C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


SLSQP-RealAmplitudes — Std Dev: 0.00990

Running VQE for: SLSQP-TwoLocal with depth 2
SLSQP-TwoLocal — Std Dev: 0.00392

Running VQE for: SLSQP-EfficientSU2 with depth 4


C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:441: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  g = append(wrapped_grad(x), 0.0)
For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 — Std Dev: 0.00005

Running VQE for: P_BFGS-RealAmplitudes with depth 4


...

P_BFGS-RealAmplitudes — Std Dev: 0.00000

Running VQE for: P_BFGS-TwoLocal with depth 2


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-TwoLocal — Std Dev: 0.00001

Running VQE for: P_BFGS-EfficientSU2 with depth 6


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 — Std Dev: 0.00000


### Print performance metrics

In [27]:
## Print the mean and standard deviation errors
print('The standard deviation in the quantum expectation values are:')
print(std_expectations_N_8)
print()

The standard deviation in the quantum expectation values are:
{'SLSQP-RealAmplitudes': {'std_expectation': 0.009895308720155864}, 'SLSQP-TwoLocal': {'std_expectation': 0.00391759558935543}, 'SLSQP-EfficientSU2': {'std_expectation': 4.766297111402442e-05}, 'P_BFGS-RealAmplitudes': {'std_expectation': 1.0984030361424827e-06}, 'P_BFGS-TwoLocal': {'std_expectation': 6.26937204472878e-06}, 'P_BFGS-EfficientSU2': {'std_expectation': 5.676417066391336e-07}}



### For a = -1, b = 2 (N = 16)

### Get the matrix and the zeromode

In [32]:
## Get the matrix and zeromode
nmax = 31
a = -1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 1.73906250e+01 -4.14983292e+01  4.16413256e+01 -2.51557647e+00
  -3.13747510e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-4.14983292e+01  1.38140625e+02 -2.04490248e+02  1.54161036e+02
  -3.10594189e+01 -2.10468228e+01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 4.16413256e+01 -2.04490248e+02  4.13640625e+02 -4.62654398e+02
   2.97161572e+02 -1.33665277e+02 -6.98044143e+01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.51557647e+00  1.54161036e+02 -4.62654398e+02  9.24890625e+02
  -7.68209032e+02  4.17083625e+02 -3.82334523e+02 -1.71932490e+02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000

C:\Users\Owner\AppData\Local\Temp\ipykernel_26540\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The value of <x^2> is:
0.6471064574160962



### Estimating the standard deviation in results (fix at 5000 optimization iterations)

In [35]:
# Initialize the optimizer -- ansatz pairings and corresponding depths
pair_depth_dict = {
    'SLSQP-RealAmplitudes': 5,
    'SLSQP-TwoLocal': 3,
    'SLSQP-EfficientSU2': 4, 
    'P_BFGS-RealAmplitudes': 5, 
    'P_BFGS-TwoLocal': 2,
    'P_BFGS-EfficientSU2': 4
}

# Run VQE
std_expectations_N_16 = performance_metrics(matrix, zeromode, pair_depth_dict, classical_expectation, nmax, L)


Running VQE for: SLSQP-RealAmplitudes with depth 5


C:\Users\Owner\AppData\Local\Temp\ipykernel_26540\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


SLSQP-RealAmplitudes — Std Dev: 0.00319

Running VQE for: SLSQP-TwoLocal with depth 3
SLSQP-TwoLocal — Std Dev: 0.00509

Running VQE for: SLSQP-EfficientSU2 with depth 4


For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 — Std Dev: 0.00286

Running VQE for: P_BFGS-RealAmplitudes with depth 5


...

P_BFGS-RealAmplitudes — Std Dev: 0.83454

Running VQE for: P_BFGS-TwoLocal with depth 2


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-TwoLocal — Std Dev: 0.00247

Running VQE for: P_BFGS-EfficientSU2 with depth 4


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 — Std Dev: 0.14336


### Print performance metrics

In [38]:
## Print the mean and standard deviation errors
print('The standard deviation in the quantum expectation values are:')
print(std_expectations_N_16)
print()

The standard deviation in the quantum expectation values are:
{'SLSQP-RealAmplitudes': {'std_expectation': 0.0031885196167281116}, 'SLSQP-TwoLocal': {'std_expectation': 0.0050880409483701474}, 'SLSQP-EfficientSU2': {'std_expectation': 0.0028558802021963715}, 'P_BFGS-RealAmplitudes': {'std_expectation': 0.8345413720384601}, 'P_BFGS-TwoLocal': {'std_expectation': 0.002472638557607819}, 'P_BFGS-EfficientSU2': {'std_expectation': 0.14336106739143378}}

